In [2]:
# Carregar pacotes necessários
from Bio import SeqIO   # Importar a função SeqIO do pacote Bio
from Bio import AlignIO   # Importar a função AlignIO do pacote Bio
from Bio import Entrez    # Importar a função Entrez do pacote Bio
from collections import defaultdict   # Importar a função defaultdict do pacote collections
import subprocess  # Importar a função subprocess
import re   # Importar a função re
import os    # Importar a função os

# Configurar o e-mail para acessar o NCBI
Entrez.email = "nathanpaivab@gmail.com"

In [ ]:


def search_genes_ncbi_range(start_id, end_id):
    """
    Pesquisa genes no NCBI a partir de um intervalo de códigos de identificação de genes.

    Parameters:
    start_id (str): Código de identificação do gene inicial no NCBI.
    end_id (str): Código de identificação do gene final no NCBI.

    Returns:
    list: Lista de SeqRecord contendo as sequências dos genes.
    """
    # Extrair prefixo e números dos IDs
    prefix = re.match(r"([A-Za-z]+)", start_id).group(1)
    start_num = int(re.match(r"[A-Za-z]+(\d+)", start_id).group(1))
    end_num = int(re.match(r"[A-Za-z]+(\d+)", end_id).group(1))

    gene_ids = [f"{prefix}{i:06d}" for i in range(start_num, end_num + 1)]

    gene_records = []
    for gene_id in gene_ids:
        handle = Entrez.efetch(db="nucleotide", id=gene_id, rettype="gb", retmode="text")
        record = SeqIO.read(handle, "genbank")
        handle.close()
        gene_records.append(record)
    return gene_records


# Exemplo de uso
start_id = "EU017577"
end_id = "EU018044"


gene_records = search_genes_ncbi_range(start_id, end_id)

# Criar pasta para armazenar as sequências
output_dir = f"{start_id}_{end_id}_sequences"
os.makedirs(output_dir, exist_ok=True)

for record in gene_records:
    # Extrair o nome da espécie e do gene do GenBank
    species_name = record.annotations.get("organism", "Unknown_species").replace(" ", "_")
    gene_name = "Unknown_gene"
    for feature in record.features:
        if feature.type == "CDS" and "gene" in feature.qualifiers:
            gene_name = feature.qualifiers["gene"][0]
            break

    # Salvar cada sequência em um arquivo separado com o formato especificado
    output_path = os.path.join(output_dir, f"{record.id}_{species_name}_{gene_name}.fasta")
    SeqIO.write(record, output_path, "fasta")
    print(f"Salvo: {output_path}")


Salvo: EU017577_EU018044_sequences/EU017577.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017578.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017579.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017580.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017581.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017582.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017583.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017584.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017585.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017586.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017587.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017588.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017589.1_Proceratophrys_boi

In [ ]:
#concatenar sequências de genes

# Diretório onde as sequências estão armazenadas
input_dir = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences"

# Dicionário para armazenar sequências por gene
gene_sequences = defaultdict(list)

# Iterar sobre os arquivos no diretório
for filename in os.listdir(input_dir):
    if filename.endswith(".fasta"):
        filepath = os.path.join(input_dir, filename)
        record = SeqIO.read(filepath, "fasta")
        gene_name = filename.split('_')[-1].replace(".fasta", "")
        gene_sequences[gene_name].append(record)

# Concatenar sequências de cada gene em um único arquivo
for gene_name, records in gene_sequences.items():
    output_path = os.path.join(input_dir, f"{gene_name}_concatenated.fasta")
    SeqIO.write(records, output_path, "fasta")
    print(f"Salvo: {output_path}")

Salvo: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_concatenated.fasta
Salvo: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_concatenated.fasta


In [ ]:
#concatenar sequências de genes

# Diretório onde as sequências estão armazenadas
input_dir = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences"

# Dicionário para armazenar sequências por gene
gene_sequences = defaultdict(list)

# Iterar sobre os arquivos no diretório
for filename in os.listdir(input_dir):
    if filename.endswith(".fasta"):
        filepath = os.path.join(input_dir, filename)
        record = SeqIO.read(filepath, "fasta")
        gene_name = filename.split('_')[-1].replace(".fasta", "")
        gene_sequences[gene_name].append(record)

# Concatenar sequências de cada gene em um único arquivo
for gene_name, records in gene_sequences.items():
    output_path = os.path.join(input_dir, f"{gene_name}_concatenated.fasta")
    SeqIO.write(records, output_path, "fasta")
    print(f"Salvo: {output_path}")

Salvo: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_concatenated.fasta


In [ ]:
#filtrar sequências de NAD2 apenas de Proceratophrys boiei


# Caminho do arquivo concatenado
input_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_concatenated.fasta"
output_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei.fasta"

# Ler as sequências do arquivo concatenado
records = list(SeqIO.parse(input_file, "fasta"))

# Filtrar sequências que pertencem à espécie Proceratophrys boiei
filtered_sequences = [record for record in records if "Proceratophrys boiei" in record.description]

# Salvar as sequências filtradas em um novo arquivo
SeqIO.write(filtered_sequences, output_file, "fasta")

print(f"Sequências filtradas salvas em: {output_file}")

Sequências filtradas salvas em: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei.fasta


In [ ]:
#filtrar sequências de cytb apenas de Proceratophrys boiei

# Caminho do arquivo concatenado
input_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_concatenated.fasta"
output_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei.fasta"

# Ler as sequências do arquivo concatenado
records = list(SeqIO.parse(input_file, "fasta"))

# Filtrar sequências que pertencem à espécie Proceratophrys boiei
filtered_sequences = [record for record in records if "Proceratophrys boiei" in record.description]

# Salvar as sequências filtradas em um novo arquivo
SeqIO.write(filtered_sequences, output_file, "fasta")

print(f"Sequências filtradas salvas em: {output_file}")

Sequências filtradas salvas em: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei.fasta


In [ ]:
#filtrar sequências de c-myc apenas de Proceratophrys boiei

# Caminho do arquivo concatenado
input_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_concatenated.fasta"
output_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei.fasta"

# Ler as sequências do arquivo concatenado
records = list(SeqIO.parse(input_file, "fasta"))

# Filtrar sequências que pertencem à espécie Proceratophrys boiei
filtered_sequences = [record for record in records if "Proceratophrys boiei" in record.description]

# Salvar as sequências filtradas em um novo arquivo
SeqIO.write(filtered_sequences, output_file, "fasta")

print(f"Sequências filtradas salvas em: {output_file}")

Sequências filtradas salvas em: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei.fasta


In [ ]:
#renomear as sequências de ND2 de Proceratophrys boiei para que fique apenas o código e o quinto termo do cabeçalho


# Caminho do arquivo de entrada e saída
input_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei.fasta"
output_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei_renamed.fasta"

# Ler as sequências do arquivo de entrada
records = list(SeqIO.parse(input_file, "fasta"))

# Renomear os cabeçalhos das sequências
for record in records:
    description_parts = record.description.split()
    sequence_id = description_parts[0]
    fifth_term = description_parts[4] if len(description_parts) > 4 else ""
    new_description = f"{sequence_id} {fifth_term}"
    record.description = new_description
    record.id = sequence_id  # Manter apenas o ID da sequência

# Salvar as sequências com os novos cabeçalhos
SeqIO.write(records, output_file, "fasta")

print(f"Sequências renomeadas salvas em: {output_file}")

Sequências renomeadas salvas em: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei_renamed.fasta


In [ ]:
#renomear as sequências de cytb de Proceratophrys boiei para que fique apenas o código e o quinto termo do cabeçalho

# Caminho do arquivo de entrada e saída
input_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei.fasta"
output_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei_renamed.fasta"

# Ler as sequências do arquivo de entrada
records = list(SeqIO.parse(input_file, "fasta"))

# Renomear os cabeçalhos das sequências
for record in records:
    description_parts = record.description.split()
    sequence_id = description_parts[0]
    fifth_term = description_parts[4] if len(description_parts) > 4 else ""
    new_description = f"{sequence_id} {fifth_term}"
    record.description = new_description
    record.id = sequence_id  # Manter apenas o ID da sequência

# Salvar as sequências com os novos cabeçalhos
SeqIO.write(records, output_file, "fasta")

print(f"Sequências renomeadas salvas em: {output_file}")

Sequências renomeadas salvas em: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei_renamed.fasta


In [ ]:
#renomear as sequências de ND2 de Proceratophrys boiei para que fique apenas o código e o quinto termo do cabeçalho

# Caminho do arquivo de entrada e saída
input_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei.fasta"
output_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei_renamed.fasta"

# Ler as sequências do arquivo de entrada
records = list(SeqIO.parse(input_file, "fasta"))

# Renomear os cabeçalhos das sequências
for record in records:
    description_parts = record.description.split()
    sequence_id = description_parts[0]
    fifth_term = description_parts[4] if len(description_parts) > 4 else ""
    new_description = f"{sequence_id} {fifth_term}"
    record.description = new_description
    record.id = sequence_id  # Manter apenas o ID da sequência

# Salvar as sequências com os novos cabeçalhos
SeqIO.write(records, output_file, "fasta")

print(f"Sequências renomeadas salvas em: {output_file}")

Sequências renomeadas salvas em: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei_renamed.fasta


In [ ]:
#caso queira as sequências de Proceratophrys boiei apenas com o número do voucher 

In [ ]:
#caso queira as sequências de Proceratophrys boiei apenas com o número do voucher 
# Caminho do arquivo de entrada e saída
input_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei_renamed.fasta"
output_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei_renamed_second_term.fasta"

# Ler as sequências do arquivo de entrada
records = list(SeqIO.parse(input_file, "fasta"))

# Renomear os cabeçalhos das sequências
for record in records:
    description_parts = record.description.split()
    second_term = description_parts[1] if len(description_parts) > 1 else ""
    new_description = f"{second_term}"
    record.description = new_description
    record.id = second_term  # Manter apenas o segundo termo como ID da sequência

# Salvar as sequências com os novos cabeçalhos
SeqIO.write(records, output_file, "fasta")

print(f"Sequências renomeadas salvas em: {output_file}")

Sequências renomeadas salvas em: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei_renamed_second_term.fasta


In [ ]:
#caso queira as sequências de Proceratophrys boiei apenas com o número do voucher 
# Caminho do arquivo de entrada e saída
input_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei_renamed.fasta"
output_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei_renamed_second_term.fasta"

# Ler as sequências do arquivo de entrada
records = list(SeqIO.parse(input_file, "fasta"))

# Renomear os cabeçalhos das sequências
for record in records:
    description_parts = record.description.split()
    second_term = description_parts[1] if len(description_parts) > 1 else ""
    new_description = f"{second_term}"
    record.description = new_description
    record.id = second_term  # Manter apenas o segundo termo como ID da sequência

# Salvar as sequências com os novos cabeçalhos
SeqIO.write(records, output_file, "fasta")

print(f"Sequências renomeadas salvas em: {output_file}")

Sequências renomeadas salvas em: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei_renamed_second_term.fasta


In [ ]:
#caso queira as sequências de Proceratophrys boiei apenas com o número do voucher 
# Caminho do arquivo de entrada e saída
input_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei_renamed.fasta"
output_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei_renamed_second_term.fasta"

# Ler as sequências do arquivo de entrada
records = list(SeqIO.parse(input_file, "fasta"))

# Renomear os cabeçalhos das sequências
for record in records:
    description_parts = record.description.split()
    second_term = description_parts[1] if len(description_parts) > 1 else ""
    new_description = f"{second_term}"
    record.description = new_description
    record.id = second_term  # Manter apenas o segundo termo como ID da sequência

# Salvar as sequências com os novos cabeçalhos
SeqIO.write(records, output_file, "fasta")

print(f"Sequências renomeadas salvas em: {output_file}")

Sequências renomeadas salvas em: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei_renamed_second_term.fasta


três arquivos com as sequências prontas para fazer o alinhamento foram organizados:

/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei_renamed_second_term.fasta
/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei_renamed_second_term.fasta
/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei_renamed_second_term.fasta

será feito o alinhamento dessas sequências

In [5]:

# Lista de arquivos de entrada e saída
input_files = [
    "/home/nathan/Documents/Doutorado_diversidade_genética/Phylogeogra/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei_renamed_second_term.fasta",
    "/home/nathan/Documents/Doutorado_diversidade_genética/Phylogeogra/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei_renamed_second_term.fasta",
    "/home/nathan/Documents/Doutorado_diversidade_genética/Phylogeogra/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei_renamed_second_term.fasta"
]

output_files = [
    "/home/nathan/Documents/Doutorado_diversidade_genética/Phylogeogra/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei_aligned.fas",
    "/home/nathan/Documents/Doutorado_diversidade_genética/Phylogeogra/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei_aligned.fas",
    "/home/nathan/Documents/Doutorado_diversidade_genética/Phylogeogra/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei_aligned.fas"
]

# Alinhar sequências usando MAFFT e salvar no formato .fas 
for input_file, output_file in zip(input_files, output_files):
    command = ["mafft", "--auto", input_file]
    with open(output_file, "w") as output_handle:
        subprocess.run(command, stdout=output_handle)
    print(f"Alinhamento concluído: {output_file}")

outputhat23=2
treein = 0
compacttree = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
All-to-all alignment.
tbfast-pair (nuc) Version 7.490
alg=L, model=DNA200 (2), 2.00 (6.00), -0.10 (-0.30), noshift, amax=0.0
0 thread(s)

outputhat23=2
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
  100 / 104
done.

Progressive alignment ... 
STEP   103 /103 
done.
tbfast (nuc) Version 7.490
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
1 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 0
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 2
sueff_global = 0.100000
nadd = 2
Loading 'hat3' ... done.
generating a scoring matrix for nucleotide (dist=200) ... done

  100 / 104
Segment   1/  1    1- 709
STEP 002-102-1  identical.    iden

Alinhamento concluído: /home/nathan/Documents/Doutorado_diversidade_genética/Phylogeogra/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei_aligned.fas


outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
All-to-all alignment.
tbfast-pair (nuc) Version 7.490
alg=L, model=DNA200 (2), 2.00 (6.00), -0.10 (-0.30), noshift, amax=0.0
0 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
   50 / 55
done.

Progressive alignment ... 
STEP    54 /54 
done.
tbfast (nuc) Version 7.490
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
1 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 0
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
nadd = 16
Loading 'hat3' ... done.
generating a scoring matrix for nucleotide (dist=200) ... done

   50 / 55
Segment   1/  1    1- 875
STEP 002-053-1  identical.    ide

Alinhamento concluído: /home/nathan/Documents/Doutorado_diversidade_genética/Phylogeogra/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei_aligned.fas


outputhat23=2
treein = 0
compacttree = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
All-to-all alignment.
tbfast-pair (nuc) Version 7.490
alg=L, model=DNA200 (2), 2.00 (6.00), -0.10 (-0.30), noshift, amax=0.0
0 thread(s)

outputhat23=2
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
  100 / 104
done.

Progressive alignment ... 
STEP   103 /103 
done.
tbfast (nuc) Version 7.490
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
1 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 0
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 2
sueff_global = 0.100000
nadd = 2
Loading 'hat3' ... done.
generating a scoring matrix for nucleotide (dist=200) ... done

  100 / 104
Segment   1/  1    1- 667
STEP 001-098-0  identical.    iden

Alinhamento concluído: /home/nathan/Documents/Doutorado_diversidade_genética/Phylogeogra/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei_aligned.fas


STEP 002-102-1  identical.   
Converged.

done
dvtditr (nuc) Version 7.490
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
0 thread(s)


Strategy:
 L-INS-i (Probably most accurate, very slow)
 Iterative refinement method (<2) with LOCAL pairwise alignment information

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more gaps into gap-rich regions than previous versions.
To disable this change, add the --leavegappyregion option.



In [7]:

# Lista de arquivos de alinhamento
alignment_files = [
    "/home/nathan/Documents/Doutorado_diversidade_genética/Phylogeogra/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei_aligned.fas",
    "/home/nathan/Documents/Doutorado_diversidade_genética/Phylogeogra/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei_aligned.fas",
    "/home/nathan/Documents/Doutorado_diversidade_genética/Phylogeogra/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei_aligned.fas"
]

# Visualizar os alinhamentos
for alignment_file in alignment_files:
    alignment = AlignIO.read(alignment_file, "fasta")
    print(f"Alinhamento do arquivo: {alignment_file}")
    print(alignment)
    print("\n")

Alinhamento do arquivo: /home/nathan/Documents/Doutorado_diversidade_genética/Phylogeogra/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei_aligned.fas
Alignment with 104 rows and 708 columns
gttaaaccccctcctttactaattaacccatatgccctatcaat...att BMD830
gttaaaccccctcctttactaattaacccatatgccctatcaat...att BC767
gttaaaccccctcctttactaattaacccatatgccctatcaat...att BC774
gttaaaccccctcctttactaattaacccatatgccctatcaat...att FC1402
gttaaaccccctcctttactaattaacccgtatgccctatcaat...att SM1577
gttaaaccccctcctttactaattaacccatatgccctatcaat...att FC1065
gttaaaccccctcctttactaattaacccatatgccctatcaat...att TAP1480
gttaaaccccctcctttactaattaacccatatgccctatcaat...att BC772
gttaaaccccctcctttactaattaacccatatgccctatcaat...att BC728
gttaaaccccctcctttactaattaacccatatgccctatcaat...att BMD840
gttaaaccccctcctttactaattaacccgtatgccctatcaat...att SM1581
gttaaaccccctcctttactaattaacccatatgccctatcaat...att BMD376
gttaaaccccctcctttactaattaacccatatgccctatcaat...att FC1404
gttaaaccccctcctttactaattaacccatatgccctatcaat...att FC1